In [160]:
#importing setup
import pandas as pd
import pathlib as path

In [161]:
# importing csv files into jupyter to analyze
school_data = pd.read_csv("schools_complete.csv")
student_data = pd.read_csv("students_complete.csv")

In [162]:
# combining the two sets into one based on common column "school_name"
dfcomplete_data = pd.merge(school_data, student_data, how = "left", on = ["school_name", "school_name"])
dfcomplete_data.head(3)

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60


# District Summary

In [163]:
# First checking to see if there is any missing data
dfcomplete_data.isnull().mean()

School ID        0.0
school_name      0.0
type             0.0
size             0.0
budget           0.0
Student ID       0.0
student_name     0.0
gender           0.0
grade            0.0
reading_score    0.0
math_score       0.0
dtype: float64

In [164]:
# Total number of unique schools
total_schools = dfcomplete_data["school_name"].nunique()

total_schools

15

In [165]:
# Total number of unique students
total_students = dfcomplete_data["student_name"].count()

type(total_students) 

numpy.int64

In [166]:
# Total budget
total_budget = dfcomplete_data["budget"].unique().sum()

total_budget

24649428

In [167]:
# Average math score
math_avg = dfcomplete_data["math_score"].mean()

math_avg

78.98537145774827

In [168]:
# Average reading score
reading_avg = dfcomplete_data["reading_score"].mean()

reading_avg

81.87784018381414

In [169]:
# % passing math (>=70%)
passing_math_count = dfcomplete_data[(dfcomplete_data["math_score"] >= 70)].count()["Student ID"]
passing_math_percentage = passing_math_count / total_students * 100

passing_math_percentage

74.9808526933878

In [170]:
# % passing reading (>=70%)
passing_reading_count = dfcomplete_data[(dfcomplete_data["reading_score"] >= 70)].count()["Student ID"]
passing_reading_percent = passing_reading_count / total_students * 100

passing_reading_percent

85.80546336482001

In [171]:
# % of students passing both math and reading (>=70%)
passing_reading_math_count = dfcomplete_data[(dfcomplete_data["math_score"] >= 70) & (dfcomplete_data["reading_score"] >= 70)].count()["Student ID"]
passing_reading_math_percent = passing_reading_math_count / total_students *100

passing_reading_math_percent

65.17232575950983

In [172]:
# Create highlevel snapshot at district level

district_summary = {"Total Schools":total_schools, 
                      "Total Students": total_students,
                      "Total Budget": total_budget,
                      "Average Math Score": math_avg,
                      "Average Reading Score": reading_avg,
                      "% Passing Math": passing_math_percentage,
                      "% Passing Reading": passing_reading_percent,
                      "% passing Both": passing_reading_math_percent
                     }




# setting the dictinary to a data frame

dfdistrict_summary = pd.DataFrame([district_summary])


dfdistrict_summary["Total Students"] = dfdistrict_summary["Total Students"].map("{:,}".format)
dfdistrict_summary["Total Budget"] = dfdistrict_summary["Total Budget"].map("${:,.2f}".format)


dfdistrict_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% passing Both
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [173]:
# setting index to school name and value associated to be type
school_types = school_data.set_index(["school_name"])["type"]

In [174]:
school_data.head(3)

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600


In [175]:
# Count to total students per school
per_school_count = school_data.groupby("school_name")["size"].sum()

In [176]:
# Calculate total school budget and per capita spending per school

per_school_budget = school_data.groupby("school_name")["budget"].sum()

per_school_capita = per_school_budget / per_school_count

In [177]:
# Calculate the average test scores per school

per_school_math = dfcomplete_data.groupby("school_name")["math_score"].mean()

per_school_reading = dfcomplete_data.groupby("school_name")["reading_score"].mean()

In [178]:
# Calculate total number to of students with passing grade per school, math

students_passing_math = dfcomplete_data[dfcomplete_data["math_score"] >= 70]

school_students_passing_math = students_passing_math.groupby("school_name")["size"].count()

In [179]:
# Calculate total number to of students with passing grade per school, reading
students_passing_reading = dfcomplete_data[dfcomplete_data["reading_score"] >= 70]

school_students_passing_reading = students_passing_reading.groupby("school_name")["size"].count()

In [180]:
# Calculate the percent of students passing both math and reading by school

students_passing_math_and_reading = dfcomplete_data[(dfcomplete_data["math_score"] >= 70) & (dfcomplete_data["reading_score"] >= 70)]

school_students_passing_math_and_reading = students_passing_math_and_reading.groupby("school_name")["size"].count()

In [181]:
# Calculate passing amounts into percents by school

per_school_passing_math = school_students_passing_math / per_school_count * 100

per_school_passing_reading = school_students_passing_reading / per_school_count * 100

overall_passing_rate = school_students_passing_math_and_reading / per_school_count * 100

In [182]:

per_school_summary = {"School Type": school_types,
                      "Total Students": per_school_count,
                      "Total School Budget": per_school_budget,
                      "Dollar Amount Per Student": per_school_capita,
                      "Average Math Score": per_school_math,
                      "Average Reading Score": per_school_reading,
                      "% Passing Math": per_school_passing_math,
                      "% Passing Reading": per_school_passing_reading,
                      "% Passing Both": overall_passing_rate 
                      } 

# formatting correct units
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Dollar Amount Per Student"] = per_school_summary["Dollar Amount Per Student"].map("${:,.2f}".format)

# calling dict to data frame
dfper_school_summary = pd.DataFrame(per_school_summary)

In [206]:
dfper_school_summary

,School Type,Total Students,Total School Budget,Dollar Amount Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Highest Performing Schools (by % passing)

In [184]:
# sort by ascending order highest % passing both subjects

top_schools = dfper_school_summary.sort_values(by = "% Passing Both", ascending = False)

# call the df with a head of 5

top_schools.head(5)

,School Type,Total Students,Total School Budget,Dollar Amount Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Bottom Performing Schools (by % passing)

In [185]:
# sort by ascending order highest % passing both subjects

bottom_schools = dfper_school_summary.sort_values(by = "% Passing Both", ascending = True)

# call the df with a head of 5

bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Dollar Amount Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores by Grade

In [186]:
# Seperate by grade

ninth_graders = dfcomplete_data[(dfcomplete_data["grade"] == "9th")]
tenth_graders = dfcomplete_data[(dfcomplete_data["grade"] == "10th")]
eleventh_graders = dfcomplete_data[(dfcomplete_data["grade"] == "11th")]
twelfth_graders = dfcomplete_data[(dfcomplete_data["grade"] == "12th")]

# Group by School name and math score

ninth_grade_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Create new data frame

math_scores_by_grade = {"9th": ninth_grade_math_scores,
                        "10th": tenth_grader_math_scores,
                        "11th": eleventh_grader_math_scores,
                        "12th": twelfth_grader_math_scores
                       }

# turning the dict into a df

dfmath_scores_by_grade = pd.DataFrame(math_scores_by_grade)

# Minor data wrangling

dfmath_scores_by_grade.index.name = None

#printing the df

dfmath_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [187]:
# Seperate by grade

ninth_graders2 = dfcomplete_data[(dfcomplete_data["grade"] == "9th")]
tenth_graders2 = dfcomplete_data[(dfcomplete_data["grade"] == "10th")]
eleventh_graders2 = dfcomplete_data[(dfcomplete_data["grade"] == "11th")]
twelfth_graders2 = dfcomplete_data[(dfcomplete_data["grade"] == "12th")]

# Group by School name and reading score

ninth_grade_reading_scores2 = ninth_graders2.groupby("school_name")["reading_score"].mean()
tenth_grader_reading_scores2 = tenth_graders2.groupby("school_name")["reading_score"].mean()
eleventh_grader_reading_scores2 = eleventh_graders2.groupby("school_name")["reading_score"].mean()
twelfth_grader_reading_scores2 = twelfth_graders2.groupby("school_name")["reading_score"].mean()

# Create new data frame

reading_scores_by_grade = {"9th": ninth_grade_reading_scores2,
                        "10th": tenth_grader_reading_scores2,
                        "11th": eleventh_grader_reading_scores2,
                        "12th": twelfth_grader_reading_scores2
                       }

# turning the dict into a df

dfreading_scores_by_grade = pd.DataFrame(reading_scores_by_grade)

# Minor data wrangling

dfreading_scores_by_grade.index.name = None

#printing the df

dfreading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [188]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [189]:
# Create a copy of the school summary since it has the "Per Student Budget"

dfschool_spending = dfper_school_summary.copy()

# Scores by School Size

In [190]:
# Establish the bins.

size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [191]:
# cutting the data into its bins 

per_school_summary["School Size"] = pd.cut(dfper_school_summary ["Total Students"], bins = size_bins, labels = labels)

# turing it into a df

df_per_school_summary = pd.DataFrame(per_school_summary)

In [193]:
df_per_school_summary

,School Type,Total Students,Total School Budget,Dollar Amount Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)


In [198]:
# Calculate averages for the desired columns.

size_math_scores = df_per_school_summary.groupby(["School Size"], observed = True)["Average Math Score"].mean()
size_reading_scores =df_per_school_summary.groupby(["School Size"], observed = True)["Average Reading Score"].mean()
size_passing_math = df_per_school_summary.groupby(["School Size"], observed = True)["% Passing Math"].mean()
size_passing_reading = df_per_school_summary.groupby(["School Size"], observed = True)["% Passing Reading"].mean()
size_overall_passing = df_per_school_summary.groupby(["School Size"], observed = True)["% Passing Both"].mean()

In [199]:
#creating the dict to hold the values

dfsize_summary = {"Average Math Score": size_math_scores,
                  "Average Reading Score": size_reading_scores,
                  "% Passing Math": size_passing_math,
                  "% passing Reading": size_passing_reading,
                  "% Passing Both": size_overall_passing,
                 }

#turning the dict into a df

dfsize_summary = pd.DataFrame(dfsize_summary)


dfsize_summary

,Average Math Score,Average Reading Score,% Passing Math,% passing Reading,% Passing Both
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


# Scores By School Type

In [205]:
# grouping school type to scores

average_math_score_by_type = dfper_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = dfper_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type =dfper_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = dfper_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = dfper_school_summary.groupby(["School Type"])["% Passing Both"].mean()

In [210]:
# creating a new dict to create the df later

type_summary = {"Average Math Score": average_math_score_by_type,
                  "Average Reading Score": average_reading_score_by_type,
                  "% Passing Math": average_percent_passing_math_by_type,
                  "% passing Reading": average_percent_passing_reading_by_type,
                  "% Passing Both": average_percent_overall_passing_by_type,
               }

# turning it into a df

dftype_summary = pd.DataFrame(type_summary)

dftype_summary

,Average Math Score,Average Reading Score,% Passing Math,% passing Reading,% Passing Both
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
